# Imports

In [1]:
from architectures.lenet5 import LeNet5
from architectures.alexnet import AlexNet
from architectures.vgg16 import VGG16
from architectures.v2_resnet import ResNet
from tensorflow.keras import models, layers
from Datasets import Datasets
from sklearn.model_selection import KFold
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import tensorflow.keras as keras
from tensorflow.keras.losses import categorical_crossentropy
import tensorflow as tf
import math

NotFoundError: /shared-libs/python3.6/py/lib/python3.6/site-packages/tensorflow/core/kernels/libtfkernel_sobol_op.so: undefined symbol: _ZN10tensorflow14kernel_factory17OpKernelRegistrar12InitInternalEPKNS_9KernelDefEN4absl12lts_2021032411string_viewESt10unique_ptrINS0_15OpKernelFactoryESt14default_deleteIS9_EE

# Load Data

In [ ]:
datasets = Datasets()
X_train, Y_train, X_test, Y_test = datasets.load_cifar10()

_, x_size, y_size, n_filters = X_train.shape
_, classes = Y_train.shape

In [ ]:
learning_rates = [(x ** 2) / 1000 if x != 0 else 0.001 for x in range(0, 11, 2)]
print(learning_rates)

[0.001, 0.004, 0.016, 0.036, 0.064, 0.1]


In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X_train, Y_train, train_size=32)

In [ ]:
models = {"Alex": AlexNet()#,
          #"LeNet": LeNet5,
          #"VGG16": VGG16,
          #"ResNet": ResNet,
          }

hyperparameters = {"learning_rates": learning_rates,
                   "epochs": 5,
                   "k_folds": 2,
                   "batch_size": 8,
                   "optimizers": [keras.optimizers.SGD, keras.optimizers.Adam]}

In [ ]:
best_model = None

# Prepare data for cross validation
kf = KFold(n_splits=hyperparameters["k_folds"]) 
kf.get_n_splits(X_train)

for name, model in models.items():
    best_model = None
    for optimizer in hyperparameters["optimizers"]:
        for learning_rate in hyperparameters["learning_rates"]:
            #Cross Validation loop
            for train_index, test_index in kf.split(X_train):

                # Build model
                current_model = model(classes=classes, optimizer=optimizer(learning_rate=learning_rate))

                if name != "ResNet":
                    X_tr = current_model.resize_images(X_train[train_index])
                    Y_tr = Y_train[train_index]
                    X_vl = current_model.resize_images(X_train[test_index])
                    Y_vl = Y_train[test_index]
                else:
                    X_tr = X_train[train_index]
                    Y_tr = Y_train[train_index]
                    X_vl = X_train[test_index]
                    Y_vl = Y_train[test_index]
                    
                # Generate extra data
                datagen = ImageDataGenerator(width_shift_range=0.1,
                                height_shift_range=0.1,
                                zoom_range=0.3,
                                fill_mode='nearest',
                                horizontal_flip = True)
                datagen.fit(X_tr)
                
                batch_size = hyperparameters["batch_size"]
                # Create history
                history = current_model.fit(datagen.flow(X_tr, Y_tr, batch_size=batch_size),
                                    validation_data=(X_vl, Y_vl),
                                    epochs=hyperparameters["epochs"], steps_per_epoch=len(X_tr)/batch_size)
                



Epoch 1/5
2/2 [==============================] - 3s 589ms/step - loss: 4.8740 - accuracy: 0.0000e+00 - val_loss: 2.7454 - val_accuracy: 0.0000e+00
Epoch 2/5
2/2 [==============================] - 0s 39ms/step - loss: 3.5396 - accuracy: 0.0625 - val_loss: 3.1262 - val_accuracy: 0.0000e+00
Epoch 3/5
2/2 [==============================] - 0s 38ms/step - loss: 3.9205 - accuracy: 0.1250 - val_loss: 3.2129 - val_accuracy: 0.0000e+00
Epoch 4/5
2/2 [==============================] - 0s 37ms/step - loss: 3.6799 - accuracy: 0.0625 - val_loss: 3.1260 - val_accuracy: 0.0000e+00
Epoch 5/5
2/2 [==============================] - 0s 41ms/step - loss: 3.9523 - accuracy: 0.0000e+00 - val_loss: 2.9905 - val_accuracy: 0.0625
Epoch 1/5
2/2 [==============================] - 3s 576ms/step - loss: 3.2381 - accuracy: 0.1250 - val_loss: 2.3355 - val_accuracy: 0.0625
Epoch 2/5
2/2 [==============================] - 0s 48ms/step - loss: 3.0946 - accuracy: 0.1875 - val_loss: 2.3247 - val_accuracy: 0.0625
Epoch 3/

KeyboardInterrupt: 

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=64d23a20-b5bc-41cc-930c-3bd7ede12b0c' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>